In [3]:
# imports
import json
from shapely.wkt import loads as wkt_loads
from PIL import Image, ImageDraw
import numpy as np
import highdicom as hd
from pydicom.dataset import Dataset
from pydicom.sr.codedict import codes
import logging

In [4]:
# file variables
input_file = "example/input/wkt.json"
output_file = "example/output/dicomseg.dcm"

In [5]:
def create_mask_from_wkt(wkt, img_width, img_height):
    """Convert WKT geometry to a binary mask."""
    geom = wkt_loads(wkt)
    mask_img = Image.new('1', (img_width, img_height), 0)
    draw = ImageDraw.Draw(mask_img)

    if geom.geom_type == "Polygon":
        exterior_coords = [(int(x), int(y)) for x, y in geom.exterior.coords]
        draw.polygon(exterior_coords, outline=1, fill=1)
    elif geom.geom_type == "MultiPolygon":
        for poly in geom:
            exterior_coords = [(int(x), int(y)) for x, y in poly.exterior.coords]
            draw.polygon(exterior_coords, outline=1, fill=1)

    return np.array(mask_img, dtype=bool)

In [6]:
def create_dicom_seg(source_dataset, mask_array, label_name, output_file):
    """Create a DICOM-SEG file from a mask array."""

    # Create segment description
    segment_description = hd.seg.SegmentDescription(
        segment_number=1,
        segment_label=label_name,
        segmented_property_category=codes.SCT.Tissue,
        segmented_property_type=codes.SCT.ConnectiveTissue,
        algorithm_type=hd.seg.SegmentAlgorithmTypeValues.MANUAL,
        tracking_id=f"Segment_{label_name}",
        tracking_uid=hd.UID()
    )

    # Ensure source dataset has required attributes
    source_dataset.PatientID = getattr(source_dataset, 'PatientID', 'ANONYMOUS')
    source_dataset.PatientName = getattr(source_dataset, 'PatientName', 'ANONYMOUS')
    source_dataset.PatientBirthDate = getattr(source_dataset, 'PatientBirthDate', '19700101')
    source_dataset.PatientSex = getattr(source_dataset, 'PatientSex', 'O')
    source_dataset.StudyDate = getattr(source_dataset, 'StudyDate', '20240102')
    source_dataset.StudyTime = getattr(source_dataset, 'StudyTime', '120000')
    source_dataset.ContentDate = getattr(source_dataset, 'ContentDate', '20240102')
    source_dataset.ContentTime = getattr(source_dataset, 'ContentTime', '120000')
    source_dataset.StudyInstanceUID = getattr(source_dataset, 'StudyInstanceUID', hd.UID())
    source_dataset.SeriesInstanceUID = getattr(source_dataset, 'SeriesInstanceUID', hd.UID())
    source_dataset.SOPInstanceUID = getattr(source_dataset, 'SOPInstanceUID', hd.UID())
    source_dataset.SOPClassUID = getattr(source_dataset, 'SOPClassUID', '1.2.840.10008.5.1.4.1.1.2')
    source_dataset.FrameOfReferenceUID = getattr(source_dataset, 'FrameOfReferenceUID', hd.UID())
    source_dataset.StudyID = getattr(source_dataset, 'StudyID', '1')
    source_dataset.SeriesNumber = getattr(source_dataset, 'SeriesNumber', '1')
    source_dataset.InstanceNumber = getattr(source_dataset, 'InstanceNumber', '1')
    source_dataset.AccessionNumber = getattr(source_dataset, 'AccessionNumber', '1')
    source_dataset.Modality = getattr(source_dataset, 'Modality', 'CT')
    source_dataset.SliceThickness = getattr(source_dataset, 'SliceThickness', 1.0)
    source_dataset.PixelSpacing = getattr(source_dataset, 'PixelSpacing', [1.0, 1.0])
    source_dataset.ImageOrientationPatient = getattr(source_dataset, 'ImageOrientationPatient', [1, 0, 0, 0, 1, 0])
    source_dataset.ImagePositionPatient = getattr(source_dataset, 'ImagePositionPatient', [0, 0, 0])
    source_dataset.SamplesPerPixel = getattr(source_dataset, 'SamplesPerPixel', 1)
    source_dataset.PhotometricInterpretation = getattr(source_dataset, 'PhotometricInterpretation', 'MONOCHROME2')

    try:
        seg_dataset = hd.seg.Segmentation(
            source_images=[source_dataset],
            pixel_array=mask_array.reshape(1, mask_array.shape[0], mask_array.shape[1]),
            segmentation_type=hd.seg.SegmentationTypeValues.BINARY,
            segment_descriptions=[segment_description],
            series_instance_uid=hd.UID(),
            series_number=1,
            instance_number=1,
            manufacturer="MANUFACTURER",
            manufacturer_model_name="MODEL",
            software_versions="1.0",
            device_serial_number="DEVICE_001",
            sop_instance_uid=hd.UID()
        )

        seg_dataset.save_as(output_file)
        logging.info(f"Successfully saved DICOM-SEG to {output_file}")
        return True

    except Exception as e:
        logging.error(f"Error creating DICOM-SEG: {e}")
        return False

In [7]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler("logfile.txt"), logging.StreamHandler()]
)

try:
    # Read input JSON file
    with open(input_file, 'r') as f:
        wkt_data = json.load(f)

    # Create dummy source dataset
    source_dataset = Dataset()
    source_dataset.Rows = 1024  # Default image size
    source_dataset.Columns = 1024

    # Process each WKT entry
    for i, entry in enumerate(wkt_data):
         mask = create_mask_from_wkt(
             entry['wkt'],
             source_dataset.Columns,
             source_dataset.Rows
         )

         create_dicom_seg(
             source_dataset,
             mask,
             entry.get('label', f'Segment_{i + 1}'),
             output_file
         )

except Exception as e:
    logging.error(f"Error processing WKT data: {e}")
    raise

/Users/nkwudalyndanwankwo/Documents/Centaur/ingestpy/python12/lib/python3.12/site-packages/highdicom/base.py:163: UserWarning: The string "ANONYMOUS" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in https://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  check_person_name(patient_name)
2025-01-02 06:24:13,318 - INFO - copy Image-related attributes from dataset "1.2.826.0.1.3680043.10.511.3.42985845710300178650228747298879451"
2025-01-02 06:24:13,320 - INFO - copy attributes of module "Specimen"
2025-01-02 06:24:13,320 - INFO - copy Patient-related attributes from dataset "1.2.826.0.1.3680043.10.511.3.42985845710300